## Data Analysis of covid_cases.csv dataset

#### Libraries

In [2]:
# Libraries

import pyspark.pandas as ps
from pyspark.sql import SparkSession

/home/system/.local/share/virtualenvs/code-1VgnQ5C9/lib/python3.9/site-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


#### Exploring the dataset

In [31]:
# Creating a spark session
spark = SparkSession.builder.appName('covid_cases').getOrCreate()

# Reading the dataset with spark
dfs = spark.read.csv('./datasets/covid_cases.csv', header=True, inferSchema=True)

# Printing the dataset size
print(f'Dataset size: \nRows: {dfs.count()} and Columns: {len(dfs.columns)}')

# Printing column names
print('\nColumn names:\n')
spark.createDataFrame(dfs.columns, 'string').show(len(dfs.columns), truncate=False)

# # Printing the schema
dfs.printSchema()

# # Printing the first 5 rows
dfs.show(5)

# # Printing the dataset size
print(dfs.toPandas().shape)

Dataset size: 
Rows: 62407 and Columns: 59

Column names:



ParseException: 
[PARSE_SYNTAX_ERROR] Syntax error at or near end of input.(line 1, pos 2)

== SQL ==
dd
--^^^
